In [3]:
import pandas as pd

In [34]:
!pip install bs4

   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ------------------------ --------------- 92.2/147.9 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 143.4/147.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 1.3 MB/s eta 0:00:00


In [45]:
import os
user = "root"
password_ = "imthebest*12"
host = "127.0.0.1"
port = "3306"
db_actions = "retail_online"
from sqlalchemy import create_engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password_}@{host}:{port}/{db_actions}')

### Create CSV-Files for the Database

#### 1. Customer Data

In [4]:
path = '../datalake/online_retail_II.xlsx'
xl = pd.ExcelFile(path)
xl.sheet_names  # see all sheet names

['Year 2009-2010', 'Year 2010-2011']

In [5]:
# CAUTION! Running this cell can take a while!

df_1 = pd.read_excel(path, sheet_name='Year 2009-2010')
df_2 = pd.read_excel(path, sheet_name='Year 2010-2011')
df = pd.concat([df_1, df_2], axis=0, join='inner', ignore_index=True)
df_copy = df.copy()

In [8]:
df = df.dropna()

In [9]:
df_customers_pivot = df.pivot_table(index="Customer ID", values="Country", aggfunc=(lambda x: x))
df_customers = pd.DataFrame({'customer_id': df_customers_pivot.index, 'country': df_customers_pivot['Country']}).reset_index()
df_customers['country'] = df_customers['country'].apply(lambda x: x[0].upper())
df_customers['country'] = df_customers['country'].apply(lambda x: 'UNITED KINGDOM' if x=='U' else x)
df_customers = df_customers[['customer_id', 'country']]
df_customers['customer_id'] = df_customers['customer_id'].astype(int)
df_customers

,customer_id,country
0,12346,UNITED KINGDOM
1,12347,ICELAND
2,12348,FINLAND
3,12349,ITALY
4,12350,NORWAY
...,...,...
5937,18283,UNITED KINGDOM
5938,18284,UNITED KINGDOM
5939,18285,UNITED KINGDOM
5940,18286,UNITED KINGDOM


In [10]:
customer_path = "../csv-files/customers_data.csv"
df_customers.to_csv(customer_path, index=False)

### 2. Orders Data

In [11]:
df_orders = df[['Invoice', 'StockCode', 'Quantity', 'InvoiceDate', 'Customer ID']]
df_orders = df_orders.rename(columns={'Invoice': 'invoice_number', 'StockCode': 'item_id',
                          'Quantity': 'quantity', 'InvoiceDate': 'invoice_date',
                          'Customer ID': 'customer_id'})
df_orders['invoice_date'] = df_orders['invoice_date'].apply(lambda x: int(x.timestamp()))
df_orders['customer_id'] = df_orders['customer_id'].astype(int)
df_orders

,invoice_number,item_id,quantity,invoice_date,customer_id
0,489434,85048,12,1259653500,13085
1,489434,79323P,12,1259653500,13085
2,489434,79323W,12,1259653500,13085
3,489434,22041,48,1259653500,13085
4,489434,21232,24,1259653500,13085
...,...,...,...,...,...
1067366,581587,22899,6,1323435000,12680
1067367,581587,23254,4,1323435000,12680
1067368,581587,23255,4,1323435000,12680
1067369,581587,22138,3,1323435000,12680


In [12]:
orders_path = "../csv-files/orders_data.csv"
df_orders.to_csv(orders_path, index=False)

### 3. Items Data

In [13]:
df_items_group = df.groupby('StockCode').agg({'Description': lambda x: ', '.join(x.unique())})
df_description = pd.DataFrame({'Description': df_items_group['Description']}).reset_index()

In [14]:
df_items = pd.merge(df_description, df[['StockCode', 'Price']], on="StockCode", how='left').drop_duplicates(subset=['StockCode'])
df_items = df_items.rename(columns={'StockCode': 'item_id', 'Price': 'price_in_GBP', 'Description': 'description'})
df_items

,item_id,description,price_in_GBP
0,10002,INFLATABLE POLITICAL GLOBE,0.850
324,10080,GROOVY CACTUS INFLATABLE,0.850
351,10109,BENDY COLOUR PENCILS,0.420
352,10120,DOGGY RUBBER,0.210
419,10125,MINI FUNKY DESIGN TAPES,0.850
...,...,...,...
822308,PADS,PADS TO MATCH ALL CUSHIONS,0.001
822327,POST,POSTAGE,18.000
824346,SP1002,KID'S CHALKBOARD/EASEL,2.950
824348,TEST001,This is a test product.,4.500


In [15]:
items_path = "../csv-files/items_data.csv"
df_items.to_csv(items_path, index=False)

#### 4. Currency Exchange Rate Data

In [16]:
import requests as re
from dotenv import load_dotenv
import os

In [26]:
load_dotenv()
API_KEY = os.getenv('api_key')

In [27]:
# Get list of available currencies
request_url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/codes"
response = re.get(url=request_url)

In [28]:
currency_codes = [result[0] for result in response.json()['supported_codes']]
currency_names = [result[1] for result in response.json()['supported_codes']]

df_codes = pd.DataFrame({'currency_code': currency_codes, 'currency_name': currency_names})

In [29]:
# Get latest exchange rates for 'GBP' (Pound Sterlin)
CURRENCY_CODE = 'GBP'
request_url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/latest/{CURRENCY_CODE}"
response = re.get(url=request_url)

In [31]:
rate_dict = response.json()['conversion_rates']
currencies = [currency for currency in rate_dict.keys()]
exchange_rates = [exchange_rate for exchange_rate in rate_dict.values()]

date = response.json()['time_last_update_unix']
currency_dates = [date for k in range(len(exchange_rates))]

df_exchange = pd.DataFrame({'currency_code': currencies, 'exchange_rate': exchange_rates, 'last_update': currency_dates})

df_currencies = pd.merge(df_codes, df_exchange, on='currency_code', how='outer')
df_currencies

,currency_code,currency_name,exchange_rate,last_update
0,AED,UAE Dirham,4.770848,1721307601
1,AFN,Afghan Afghani,92.056466,1721307601
2,ALL,Albanian Lek,119.357177,1721307601
3,AMD,Armenian Dram,504.465319,1721307601
4,ANG,Netherlands Antillian Guilder,2.325342,1721307601
...,...,...,...,...
157,XPF,CFP Franc,141.784689,1721307601
158,YER,Yemeni Rial,325.615482,1721307601
159,ZAR,South African Rand,23.625379,1721307601
160,ZMW,Zambian Kwacha,34.200329,1721307601


In [32]:
currencies_path = "../csv-files/currency_exchange_data.csv"
df_currencies.to_csv(currencies_path, index=False)

#### 5. Country Courrency Data

In [35]:
from bs4 import BeautifulSoup

In [37]:
# request html code
request_url = "https://www.iban.com/currency-codes"
response = re.get(request_url)

# parse html code
soup = BeautifulSoup(response.content, "html.parser")

In [38]:
result = soup.find(name='table', class_='table table-bordered downloads tablesorter')
items = result.find_all(name='tr')

In [39]:
country_names = []
currency_codes = []
for k in range(len(items)-1):
    k += 1
    item = str(items[k]).split('</td>')
    country_name = item[0][9:]
    currency_code = item[2][-3:]
    country_names.append(country_name)
    currency_codes.append(currency_code)

In [40]:
df_countries_0 = pd.DataFrame({'country': country_names, 'currency_code': currency_codes})
df_countries_group = df_countries_0.groupby('country').agg({'currency_code': lambda x: ', '.join(x.unique())})
df_countries = pd.DataFrame({'currency_code': df_countries_group['currency_code']}).reset_index()
df_countries['country'] = df_countries['country'].apply(lambda x: 'UNITED KINGDOM' if x=='UNITED KINGDOM OF GREAT    BRITAIN AND NORTHERN IRELAND (THE)' else x)
df_countries['country'] = df_countries['country'].apply(lambda x: 'UNITED ARAB EMIRATES ' if x=='UNITED ARAB EMIRATES (THE)' else x)

In [41]:
countries_path = "../csv-files/currency_country_data.csv"
df_countries.to_csv(countries_path, index=False)

### DATAFRAMES OVERVIEW

In [25]:
df_customers.head(3)

,customer_id,country
0,12346,UNITED KINGDOM
1,12347,ICELAND
2,12348,FINLAND


In [26]:
df_customers.dtypes

customer_id     int32
country        object
dtype: object

In [27]:
df_orders.head()

,invoice_number,item_id,quantity,invoice_date,customer_id
0,489434,85048,12,1259653500,13085
1,489434,79323P,12,1259653500,13085
2,489434,79323W,12,1259653500,13085
3,489434,22041,48,1259653500,13085
4,489434,21232,24,1259653500,13085


In [28]:
df_orders.dtypes

invoice_number    object
item_id           object
quantity           int64
invoice_date       int64
customer_id        int32
dtype: object

In [29]:
df_items.head(3)

,item_id,description,price_in_GBP
0,10002,INFLATABLE POLITICAL GLOBE,0.85
324,10080,GROOVY CACTUS INFLATABLE,0.85
351,10109,BENDY COLOUR PENCILS,0.42


In [30]:
df_items.dtypes

item_id          object
description      object
price_in_GBP    float64
dtype: object

In [31]:
df_currencies.head(3)

,currency_code,currency_name,exchange_rate,last_update
0,AED,UAE Dirham,4.777758,1721239201
1,AFN,Afghan Afghani,91.946611,1721239201
2,ALL,Albanian Lek,119.290066,1721239201


In [32]:
df_currencies.dtypes

currency_code     object
currency_name     object
exchange_rate    float64
last_update        int64
dtype: object

In [33]:
df_countries.head(3)

,country,currency_code
0,AFGHANISTAN,AFN
1,ALBANIA,ALL
2,ALGERIA,DZD


In [34]:
df_countries.dtypes

country          object
currency_code    object
dtype: object

### EXAMPLE MERGES AND QUERIES

In [35]:
# connect countries with currencies
pd.merge(df_countries, df_currencies, on='currency_code', how='left').dropna()

,country,currency_code,currency_name,exchange_rate,last_update
0,AFGHANISTAN,AFN,Afghan Afghani,91.946611,1.721239e+09
1,ALBANIA,ALL,Albanian Lek,119.290066,1.721239e+09
2,ALGERIA,DZD,Algerian Dinar,174.447638,1.721239e+09
3,AMERICAN SAMOA,USD,United States Dollar,1.300983,1.721239e+09
4,ANDORRA,EUR,Euro,1.190116,1.721239e+09
...,...,...,...,...,...
248,WESTERN SAHARA,MAD,Moroccan Dirham,12.782371,1.721239e+09
249,YEMEN,YER,Yemeni Rial,324.531971,1.721239e+09
250,ZAMBIA,ZMW,Zambian Kwacha,33.953177,1.721239e+09
251,ZIMBABWE,ZWL,Zimbabwean Dollar,18.261200,1.721239e+09


In [36]:
# connect customers with currencies
pd.merge(df_customers, df_countries, on='country', how='left')

,customer_id,country,currency_code
0,12346,UNITED KINGDOM,GBP
1,12347,ICELAND,ISK
2,12348,FINLAND,EUR
3,12349,ITALY,EUR
4,12350,NORWAY,NOK
...,...,...,...
5937,18283,UNITED KINGDOM,GBP
5938,18284,UNITED KINGDOM,GBP
5939,18285,UNITED KINGDOM,GBP
5940,18286,UNITED KINGDOM,GBP


In [37]:
# connect cusomters with exchange rates
pd.merge(pd.merge(df_customers, df_countries, on='country', how='left'), df_currencies, on='currency_code', how='left')

,customer_id,country,currency_code,currency_name,exchange_rate,last_update
0,12346,UNITED KINGDOM,GBP,Pound Sterling,1.000000,1.721239e+09
1,12347,ICELAND,ISK,Icelandic Króna,177.871667,1.721239e+09
2,12348,FINLAND,EUR,Euro,1.190116,1.721239e+09
3,12349,ITALY,EUR,Euro,1.190116,1.721239e+09
4,12350,NORWAY,NOK,Norwegian Krone,13.976619,1.721239e+09
...,...,...,...,...,...,...
5937,18283,UNITED KINGDOM,GBP,Pound Sterling,1.000000,1.721239e+09
5938,18284,UNITED KINGDOM,GBP,Pound Sterling,1.000000,1.721239e+09
5939,18285,UNITED KINGDOM,GBP,Pound Sterling,1.000000,1.721239e+09
5940,18286,UNITED KINGDOM,GBP,Pound Sterling,1.000000,1.721239e+09


In [38]:
# get items with highest revenue
df_orders_price = pd.merge(df_orders[['item_id', 'quantity']], df_items[['item_id', 'price_in_GBP']], on='item_id', how='left')
df_orders_price['total_price'] = df_orders_price['quantity'] * df_orders_price['price_in_GBP']
df_orders_price.pivot_table(index='item_id', values='total_price', aggfunc=sum).sort_values(by="total_price", ascending=False).head()

,total_price
item_id,
22423,298936.50
85123A,265691.75
85099B,182843.70
84879,134243.46
47566,108507.75


In [48]:
df_currencies['last_update'].dtype

dtype('int64')

#POPULATING TABLES IN MYSQL SCHEMA(retail_online) FROM DATABASES

In [53]:
df_countries.shape

(253, 2)

253

In [ ]:
df_currencies.to_sql(name='currencies', con=engine, index=False, if_exists='append')

df_countries.to_sql(name='countries', con=engine, index=False, if_exists='append')

In [60]:
df_customers.to_sql(name='customers', con=engine, index=False, if_exists='append')

5942

In [61]:
df_items.to_sql(name='items', con=engine, index=False, if_exists='append')

4646

In [65]:
df_orders.to_sql(name='orders', con=engine, index=False, if_exists='append')

824364